# DATA

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# 1. PORTFOLIO CONFIGURATION
# We define a diversified set of tickers: Quality/Intl Factors, Emerging Markets,
# Precious Metals, Managed Futures (DBMF), Crypto, and the S&P 500.
tickers = ["QUAL", "IQLT", "EMGF", "GLTR", "DBMF", "ARGT", "BTC-USD", "ETH-USD", "VOO"]
print(f"Initializing data retrieval for {len(tickers)} assets...")

# 2. DATA DOWNLOAD (HISTORICAL PRICES)
# 'period="max"' retrieves the entire price history from each ticker's inception date.
# We focus on the 'Close' price for simplicity in calculating returns.
data = yf.download(tickers, period="max", progress=False)['Close']

# 3. DATA AUDIT AND PROCESSING
# We create a list to store time-series metrics for each asset.
report_data = []

for ticker in tickers:
    if ticker in data.columns:
        # We drop NaN values to find the specific active trading window for each ticker
        series = data[ticker].dropna()

        if not series.empty:
            start_date = series.index.min()
            end_date = series.index.max()
            trading_days = len(series)

            # Estimate years based on the standard ~252 trading days per year
            years_of_history = trading_days / 252

            report_data.append({
                'Ticker': ticker,
                'Start Date': start_date.date(),
                'End Date': end_date.date(),
                'Days Logged': trading_days,
                'Years (Approx)': round(years_of_history, 2)
            })

# 4. STRUCTURED REPORT GENERATION
# Convert the list to a DataFrame for clean visualization and sort by Start Date.
df_report = pd.DataFrame(report_data).sort_values(by='Start Date')

print("\n" + "="*60)
print("             HISTORICAL DATA AVAILABILITY REPORT")
print("="*60)
print(df_report.to_string(index=False))

# 5. OPTIMIZATION WINDOW ANALYSIS
# This is critical: To perform portfolio optimization
# all assets must share the same date range.
# Using dropna() on the entire DataFrame finds the "intersection" start date.
common_start_date = data.dropna().index.min().date()

print("\n" + "-"*60)
print(f"MULTI-ASSET OPTIMIZATION ANALYSIS:")
print(f"To analyze all assets simultaneously, your lookback period")
print(f"must begin at the latest inception date: {common_start_date}")
print("-"*60)

Initializing data retrieval for 9 assets...


/tmp/ipython-input-106282805.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period="max", progress=False)['Close']



             HISTORICAL DATA AVAILABILITY REPORT
 Ticker Start Date   End Date  Days Logged  Years (Approx)
    VOO 2010-09-09 2025-12-31         3852           15.29
   GLTR 2010-10-22 2025-12-31         3821           15.16
   ARGT 2011-03-03 2025-12-31         3731           14.81
   QUAL 2013-07-18 2025-12-31         3134           12.44
BTC-USD 2014-09-17 2026-01-01         4125           16.37
   IQLT 2015-01-21 2025-12-31         2754           10.93
   EMGF 2015-12-18 2025-12-31         2523           10.01
ETH-USD 2017-11-09 2026-01-01         2976           11.81
   DBMF 2019-05-08 2025-12-31         1673            6.64

------------------------------------------------------------
MULTI-ASSET OPTIMIZATION ANALYSIS:
To analyze all assets simultaneously, your lookback period
must begin at the latest inception date: 2019-05-08
------------------------------------------------------------


In [ ]:
# 1. Formatting and Cleaning
pd.options.display.float_format = '{:,.2f}'.format
data = data.dropna()
data.index = pd.to_datetime(data.index)

# --- Output Results ---
print("DATASET PREVIEW: ASSETS IN USD")
print("=" * 85)
print(f"Showing start and end of the converted dataset:\n")

# Display first and last 5 rows
display(data.head())
print("\n" + "."*85 + "\n")
display(data.tail())
print("=" * 85)

DATASET PREVIEW: ASSETS IN USD
Showing start and end of the converted dataset:



Ticker,ARGT,BTC-USD,DBMF,EMGF,ETH-USD,GLTR,IQLT,QUAL,VOO
Date,,,,,,,,,
2019-05-08,25.20,"5,982.46",17.21,33.17,170.95,62.75,24.83,81.74,237.85
2019-05-09,25.38,"6,174.53",17.22,32.64,170.29,62.66,24.62,81.48,237.12
2019-05-10,25.40,"6,378.85",17.22,32.70,173.14,63.11,24.79,81.85,238.23
2019-05-13,24.73,"7,814.92",17.10,31.59,196.85,63.25,24.33,79.86,232.39
2019-05-14,25.07,"7,994.42",17.19,31.96,217.15,63.30,24.57,80.49,234.40



.....................................................................................



Ticker,ARGT,BTC-USD,DBMF,EMGF,ETH-USD,GLTR,IQLT,QUAL,VOO
Date,,,,,,,,,
2025-12-24,91.48,"87,611.96",29.44,57.46,"2,945.59",212.80,45.72,200.99,634.78
2025-12-26,92.17,"87,301.43",29.63,57.92,"2,925.75",222.35,45.80,201.03,634.84
2025-12-29,91.68,"87,138.14",29.28,57.88,"2,934.54",208.96,45.61,200.50,632.60
2025-12-30,91.46,"88,430.13",28.14,57.98,"2,971.42",212.25,45.68,200.10,631.72
2025-12-31,91.41,"87,508.83",28.06,57.85,"2,967.04",205.60,45.45,198.62,627.13


Metrics

In [ ]:
from scipy.stats import norm

# --- 1. CONFIGURATION ---
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)  # Ensures no column skipping
pd.set_option('display.width', 1000)        # Prevents line breaks
TRADING_DAYS = 252
rf_rate = 0.03

# --- 2. DATA PROCESSING ---
returns = np.log(data / data.shift(1)).dropna()

def get_asset_metrics(ticker):
    series = returns[ticker]
    prices = data[ticker]

    # Performance & Volatility
    ann_ret = series.mean() * TRADING_DAYS
    ann_vol = series.std() * np.sqrt(TRADING_DAYS)

    # Sharpe Ratio in Basis Points (bps) - Fixed calculation
    sharpe_bps = ((ann_ret - rf_rate)/ ann_vol) * 100

    # --- DETAILED DRAWDOWN ANALYSIS ---
    roll_max = prices.cummax()
    drawdowns = (prices - roll_max) / roll_max
    max_dd = drawdowns.min()

    # Key Dates
    trough_date = drawdowns.idxmin()
    peak_date = prices[:trough_date].idxmax()

    # Phase 1: Contraction (Peak to Trough)
    days_falling = (trough_date - peak_date).days

    # Phase 2: Recovery (Trough to previous Peak)
    post_trough_prices = prices[trough_date:]
    recovery_series = post_trough_prices[post_trough_prices >= prices[peak_date]]

    if not recovery_series.empty:
        recovery_date = recovery_series.index[0]
        days_recovering = (recovery_date - trough_date).days
        recovery_status = recovery_date.date()
    else:
        recovery_status = "Ongoing"
        days_recovering = (prices.index[-1] - trough_date).days

# --- NON-PARAMETRIC RISK (Historical Simulation - 1 Day) ---
    def calc_hist_risk(series, confidence_level):
        # We look for the alpha percentile (e.g., 1% or 5%)
        alpha = 1 - confidence_level

        # VaR: The empirical quantile
        # We report it as a positive magnitude of loss
        var_1d = -np.percentile(series, alpha * 100)

        # CVaR: The average of losses exceeding the VaR
        tail_losses = series[series <= -var_1d]
        cvar_1d = -tail_losses.mean()

        return var_1d, cvar_1d

    # Calculate 1-Day Historical Metrics
    v95_1d, cv95_1d = calc_hist_risk(series, 0.95)
    v99_1d, cv99_1d = calc_hist_risk(series, 0.99)

    return {
        'Ticker': ticker,
        'Ann_Return %': ann_ret * 100,
        'Ann_Vol %': ann_vol * 100,
        'Sharpe_bps': sharpe_bps,
        'Max_DD %': max_dd * 100,
        'Days_Fall': days_falling,
        'Days_Recov': days_recovering,
        'Peak_Date': peak_date.date(),
        'Trough_Date': trough_date.date(),
        'Recov_Date': recovery_status,
        'VaR_95 %': v95_1d * 100,
        'CVaR_95 %': cv95_1d * 100,
        'VaR_99 %': v99_1d * 100,
        'CVaR_99 %': cv99_1d * 100
    }

# --- 3. EXECUTION ---
metrics_results = [get_asset_metrics(t) for t in data.columns]
df_report = pd.DataFrame(metrics_results).set_index('Ticker')

# Grouped Columns for horizontal consistency
perf_risk_cols = ['Ann_Return %', 'Ann_Vol %', 'Sharpe_bps', 'VaR_95 %', 'CVaR_95 %', 'VaR_99 %', 'CVaR_99 %']
drawdown_cols = ['Max_DD %', 'Days_Fall', 'Days_Recov', 'Peak_Date', 'Trough_Date', 'Recov_Date']

# --- 4. PROFESSIONAL CONSOLE OUTPUT ---
print("\n" + "="*130)
print("ASSET REPORT: PERFORMANCE, RISK & DRAWDOWN ANALYSIS")
print("="*130)

print("\n SECTION 1: PERFORMANCE & TAIL RISK PROFILE")
print("-" * 130)
print(df_report[perf_risk_cols])



print("\n SECTION 2: DRAWDOWN DYNAMICS (CONTRACTION & RECOVERY)")
print("-" * 130)
print(df_report[drawdown_cols])



print("\n SECTION 3: CROSS-ASSET CORRELATION MATRIX (%)")
print("-" * 130)
print(returns.corr() * 100)

print("\n" + "="*130)
print(f"Report Period: {returns.index[0].date()} to {returns.index[-1].date()} | Trading Days: {TRADING_DAYS}")
print("="*130)


ASSET REPORT: PERFORMANCE, RISK & DRAWDOWN ANALYSIS

 SECTION 1: PERFORMANCE & TAIL RISK PROFILE
----------------------------------------------------------------------------------------------------------------------------------
         Ann_Return %  Ann_Vol %  Sharpe_bps  VaR_95 %  CVaR_95 %  VaR_99 %  CVaR_99 %
Ticker                                                                                
ARGT            19.42      34.84       47.13      3.13       5.00      5.39       9.45
BTC-USD         40.44      63.97       58.53      5.77       9.36     11.29      16.79
DBMF             7.37      12.38       35.29      1.31       1.94      2.32       3.23
EMGF             8.38      19.80       27.19      1.79       2.85      3.18       5.38
ETH-USD         43.01      83.07       48.17      7.35      12.32     14.32      21.96
GLTR            17.89      20.07       74.17      1.85       2.93      3.35       4.95
IQLT             9.11      18.51       33.02      1.61       2.68      3.00